In [ ]:
import pandas as pd
import plotly.express as px
from copy import copy
from scipy import stats
import matplotlib.pyplot as plt
import numpy as np
import plotly.figure_factory as ff
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from tensorflow import keras

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
df=pd.read_csv('/content/CGPOWER.NS.csv')
df

,Date,Open,High,Low,Close,Adj Close,Volume
0,2002-07-01,0.213810,0.223708,0.212113,0.218335,0.021701,18468968.0
1,2002-07-02,0.218052,0.223143,0.213244,0.214092,0.021279,14604782.0
2,2002-07-03,0.214092,0.216921,0.210981,0.211830,0.021054,3852022.0
3,2002-07-04,0.213810,0.232758,0.213810,0.227102,0.022572,50100446.0
4,2002-07-05,0.225688,0.238415,0.223708,0.235304,0.023387,29450709.0
...,...,...,...,...,...,...,...
4662,2021-03-19,54.700001,59.900002,54.700001,58.950001,58.950001,15333373.0
4663,2021-03-22,61.400002,61.849998,59.549999,61.849998,61.849998,3119312.0
4664,2021-03-23,63.000000,64.900002,63.000000,64.900002,64.900002,1832572.0
4665,2021-03-24,68.099998,68.099998,65.699997,68.000000,68.000000,10503499.0


In [ ]:
def interactive_plot(df, title):
  fig = px.line(title = title)
  for i in df.columns[1:]:
    fig.add_scatter(x = df['Date'], y = df[i], name = i)
  fig.show()

In [ ]:
df=df.sort_values(by=['Date'])
df=df.dropna()
df1=np.array(df['Close'])
len(df1)
df0=pd.DataFrame({'Date':df['Date'][:],'Close':df['Close'][:]})
df0

,Date,Close
0,2002-07-01,0.218335
1,2002-07-02,0.214092
2,2002-07-03,0.211830
3,2002-07-04,0.227102
4,2002-07-05,0.235304
...,...,...
4662,2021-03-19,58.950001
4663,2021-03-22,61.849998
4664,2021-03-23,64.900002
4665,2021-03-24,68.000000


In [ ]:
interactive_plot(df0,'plot')

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler(feature_range=(0,1))
df1=scaler.fit_transform(np.array(df['Close']).reshape(-1,1))

In [ ]:
df1

array([[5.80502528e-04],
       [5.44662990e-04],
       [5.25556454e-04],
       ...,
       [5.46930029e-01],
       [5.73114921e-01],
       [5.65512833e-01]])

In [ ]:

##splitting dataset into train and test split
training_size=int(len(df1)*0.65)
test_size=len(df1)-training_size
train_data,test_data=df1[0:training_size,:],df1[training_size:len(df1),:1]

In [ ]:
def create_dataset(dataset, time_step=1):
	dataX, dataY = [], []
	for i in range(len(dataset)-time_step-1):
		a = dataset[i:(i+time_step), 0]   ###i=0, 0,1,2,3-----99   100
		dataX.append(a)
		dataY.append(dataset[i + time_step, 0])
	return np.array(dataX), np.array(dataY)

In [ ]:
time_step=100
X_train, y_train = create_dataset(train_data, time_step)
X_test, y_test = create_dataset(test_data, time_step)

In [ ]:
print(X_test.shape), print(y_test.shape)


(1528, 100)
(1528,)


(None, None)

In [ ]:

# reshape input to be [samples, time steps, features] which is required for LSTM
X_train =X_train.reshape(X_train.shape[0],X_train.shape[1] , 1)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1] , 1)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM

In [ ]:
model=Sequential()
model.add(LSTM(50,return_sequences=True,input_shape=(100,1)))
model.add(LSTM(50,return_sequences=True))
model.add(LSTM(50))
model.add(Dense(1))
model.compile(loss='mean_squared_error',optimizer='adam')

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 100, 50)           10400     
                                                                 
 lstm_1 (LSTM)               (None, 100, 50)           20200     
                                                                 
 lstm_2 (LSTM)               (None, 50)                20200     
                                                                 
 dense (Dense)               (None, 1)                 51        
                                                                 
Total params: 50,851
Trainable params: 50,851
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=100,batch_size=64,verbose=1)

Epoch 1/100
46/46 [==============================] - 14s 46ms/step - loss: 0.0186 - val_loss: 0.0018
Epoch 2/100
46/46 [==============================] - 1s 16ms/step - loss: 0.0013 - val_loss: 0.0017
Epoch 3/100
46/46 [==============================] - 1s 16ms/step - loss: 0.0011 - val_loss: 0.0015
Epoch 4/100
46/46 [==============================] - 1s 16ms/step - loss: 0.0010 - val_loss: 0.0013
Epoch 5/100
46/46 [==============================] - 1s 16ms/step - loss: 9.4378e-04 - val_loss: 0.0014
Epoch 6/100
46/46 [==============================] - 1s 16ms/step - loss: 8.1011e-04 - val_loss: 0.0011
Epoch 7/100
46/46 [==============================] - 1s 17ms/step - loss: 7.6828e-04 - val_loss: 0.0012
Epoch 8/100
46/46 [==============================] - 1s 17ms/step - loss: 7.8816e-04 - val_loss: 0.0011
Epoch 9/100
46/46 [==============================] - 1s 17ms/step - loss: 7.3417e-04 - val_loss: 0.0010
Epoch 10/100
46/46 [==============================] - 1s 20ms/step - loss: 6.87

In [ ]:
import tensorflow as tf

In [ ]:
tf.__version__

'2.11.0'

In [ ]:
train_predict=model.predict(X_train)
test_predict=model.predict(X_test)


48/48 [==============================] - 0s 8ms/step


In [ ]:
train_predict=scaler.inverse_transform(train_predict)
test_predict=scaler.inverse_transform(test_predict)

In [ ]:
import math
from sklearn.metrics import mean_squared_error
math.sqrt(mean_squared_error(y_train,train_predict))

49.607979484594594

In [ ]:
math.sqrt(mean_squared_error(y_test,test_predict))

57.84990132830863

In [ ]:
train_predict

array([[ 0.16048266],
       [ 0.16229233],
       [ 0.16390541],
       ...,
       [71.014565  ],
       [67.69726   ],
       [68.54661   ]], dtype=float32)

In [ ]:
len(test_data)

1629

In [ ]:
x_input=test_data[len(test_data)-100:].reshape(1,-1)
x_input.shape

(1, 100)

In [ ]:
temp_input=list(x_input)
temp_input=temp_input[0].tolist()

In [ ]:
temp_input

In [ ]:
from numpy import array

lst_output=[]
n_steps=100
i=0
while(i<100):

    if(len(temp_input)>100):
        #print(temp_input)
        x_input=np.array(temp_input[1:])
        print("{} day input {}".format(i,x_input))
        x_input=x_input.reshape(1,-1)
        x_input = x_input.reshape((1, n_steps, 1))
        #print(x_input)
        yhat = model.predict(x_input, verbose=0)
        print("{} day output {}".format(i,yhat))
        temp_input.extend(yhat[0].tolist())
        temp_input=temp_input[1:]
        #print(temp_input)
        lst_output.extend(yhat.tolist())
        i=i+1
    else:
        x_input = x_input.reshape((1, n_steps,1))
        yhat = model.predict(x_input, verbose=0)
        print(yhat[0])
        temp_input.extend(yhat[0].tolist())
        print(len(temp_input))
        lst_output.extend(yhat.tolist())
        i=i+1


print(lst_output)

In [ ]:
a=['Close']
lst_output1=pd.DataFrame(scaler.inverse_transform(lst_output),columns=list(a))
lst_output1

,Close
0,83.389388
1,83.980492
2,84.573437
3,85.168055
4,85.764247
...,...
95,121.574074
96,121.664087
97,121.750289
98,121.832794


In [ ]:
my_df=pd.DataFrame({'Date':['2021-03-26','2021-03-30','2021-03-31','2021-04-01','2021-04-02','2021-04-05','2021-04-06','2021-04-07','2021-04-08','2021-04-09','2021-04-12','2021-04-13','2021-04-14','2021-04-15','2021-04-16','2021-04-19','2021-04-20','2021-04-21','2021-04-22','2021-04-23','2021-04-26','2021-04-27','2021-04-28','2021-04-29','2021-04-30','2021-05-03','2021-05-04','2021-05-05','2021-05-06','2021-05-07']})
my_df['Close']=lst_output1
my_df

,Date,Close
0,2021-03-26,83.389388
1,2021-03-30,83.980492
2,2021-03-31,84.573437
3,2021-04-01,85.168055
4,2021-04-02,85.764247
5,2021-04-05,86.361849
6,2021-04-06,86.960751
7,2021-04-07,87.560788
8,2021-04-08,88.161785
9,2021-04-09,88.763621


In [ ]:
df2=df0.append(my_df)
df2

,Date,Close
0,2002-07-01,0.218335
1,2002-07-02,0.214092
2,2002-07-03,0.211830
3,2002-07-04,0.227102
4,2002-07-05,0.235304
...,...,...
25,2021-05-03,98.325929
26,2021-05-04,98.906258
27,2021-05-05,99.482839
28,2021-05-06,100.055377


In [ ]:
interactive_plot(df2,'MY_PREDICTION')

In [ ]:


# Extract the daily energy values as a numpy array
S_Energy = df["Solar Energy"].values

In [ ]:
# Normalize the data using MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
S_Energy = scaler.fit_transform(S_Energy.reshape(-1, 1))

In [ ]:
# Split the data into training and testing sets
train_size = int(len(S_Energy) * 0.8)
test_size = len(S_Energy) - train_size
train_data, test_data = S_Energy[0:train_size, :], S_Energy[train_size:len(S_Energy), :]

In [ ]:
# Define the function to create the LSTM model
def create_model(train_data):
    X, y = [], []
    n_input=30
    for i in range(len(train_data)-n_input):
        X.append(train_data[i:i+n_input, 0])
        y.append(train_data[i+n_input, 0])
    X, y = np.array(X), np.array(y)
    X = np.reshape(X, (X.shape[0], X.shape[1], 1))
    model = Sequential()
    model.add(LSTM(50, input_shape=(n_input, 1)))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    model.fit(X, y, epochs=5, batch_size=1, verbose=2, validation_split=0.1)
    return model

In [ ]:
# Create the model using the training data
model = create_model(train_data)


In [ ]:
# Predict the solar S_Energy for the next n_input days using the testing data
n_input=30
inputs = S_Energy[len(S_Energy) - len(test_data) - n_input:]
# print(inputs)
inputs = inputs.reshape(-1,1)
# print(inputs)
inputs = scaler.transform(inputs)
print(inputs)
X_test = []
# y_test = []
for i in range(n_input, inputs.shape[0]):
    X_test.append(inputs[i-n_input:i, 0])
X_test = np.array(X_test)
# print(X_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

# model.predict(X_test)
predicted_S_Energy = model.predict(X_test)
# print(predicted_S_Energy)

predicted_S_Energy = scaler.inverse_transform(predicted_S_Energy)
# print(predicted_S_Energy)
# print(y_test)

In [ ]:
# print(len(S_Energy) - len(test_data) - n_input)
# print(len(train_data))
# print(predicted_S_Energy)
# print(df["Solar Energy"][len(S_Energy) - len(test_data):])
print(X_test)

In [ ]:
# predicted_energy=predicted_energy-100
# Plot the predicted solar energy for the next n_input days
import matplotlib.pyplot as plt
plt.plot(predicted_S_Energy)
plt.title("Predicted Solar Energy for Next n_input Days")
plt.xlabel("Days")
plt.ylabel("Energy (kWh)")
plt.show()


In [ ]:
S_Energy = df["Solar Energy"].values
train_size = int(len(S_Energy) * 0.8)

test_size = len(S_Energy) - train_size
test_data = S_Energy[train_size:len(S_Energy)]

In [ ]:
import matplotlib.pyplot as plt

# Plot the actual values

plt.plot(test_data, label='Actual')

# Plot the predicted values on the training data
plt.plot(predicted_S_Energy, label='Training Predictions')

# Plot the predicted values on the testing data
# plt.plot(test_pred, label='Testing Predictions')

# Set the title, x-label and y-label of the plot
plt.title('Actual vs Predicted')
plt.xlabel('Time')
plt.ylabel('Value')

# Display the legend and the plot
plt.legend()
plt.show()


In [ ]:
# Extract the daily energy values as a numpy array
W_Energy = df["wind Energy"].values

In [ ]:
# Normalize the data using MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
W_Energy = scaler.fit_transform(W_Energy.reshape(-1, 1))

In [ ]:
# Split the data into training and testing sets
train_size1 = int(len(W_Energy) * 0.8)
test_size1 = len(W_Energy) - train_size1
train_data1, test_data1 = W_Energy[0:train_size1, :], W_Energy[train_size1:len(W_Energy), :]

In [ ]:
# Define the function to create the LSTM model
def create_model(train_data1):
    X, y = [], []
    n_input=30
    for i in range(len(train_data1)-n_input):
        X.append(train_data1[i:i+n_input, 0])
        y.append(train_data1[i+n_input, 0])
    X, y = np.array(X), np.array(y)
    X = np.reshape(X, (X.shape[0], X.shape[1], 1))
    model = Sequential()
    model.add(LSTM(50, input_shape=(n_input, 1)))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    model.fit(X, y, epochs=5, batch_size=1, verbose=2, validation_split=0.1)
    return model

In [ ]:
# Create the model using the training data
model = create_model(train_data1)
n_input=30

In [ ]:
# Predict the solar S_Energy for the next n_input days using the testing data
inputs = W_Energy[len(W_Energy) - len(test_data1) - n_input:]
inputs = inputs.reshape(-1,1)
inputs = scaler.transform(inputs)
X_test = []
# y_test = []
for i in range(n_input, inputs.shape[0]):
    X_test.append(inputs[i-n_input:i, 0])
X_test = np.array(X_test)

X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
# print(X_test)
model.predict(X_test)
predicted_W_Energy = model.predict(X_test)
# print(predicted_W_Energy)

predicted_W_Energy = scaler.inverse_transform(predicted_W_Energy)
# print(predicted_W_Energy)
# print(y_test)

In [ ]:
# predicted_energy=predicted_energy-100
# Plot the predicted solar energy for the next n_input days
import matplotlib.pyplot as plt
plt.plot(predicted_W_Energy)
plt.title("Predicted Solar Energy for Next n_input Days")
plt.xlabel("Days")
plt.ylabel("Energy (kWh)")
plt.show()


In [ ]:
W_Energy = df["wind Energy"].values
train_size = int(len(W_Energy) * 0.8)
test_size = len(W_Energy) - train_size
test_data = W_Energy[train_size:len(W_Energy)]

In [ ]:
import matplotlib.pyplot as plt

# Plot the actual values

plt.plot(test_data, label='Actual')

# Plot the predicted values on the training data
plt.plot(predicted_W_Energy, label='Training Predictions')

# Plot the predicted values on the testing data
# plt.plot(test_pred, label='Testing Predictions')

# Set the title, x-label and y-label of the plot
plt.title('Actual vs Predicted')
plt.xlabel('Time')
plt.ylabel('Value')

# Display the legend and the plot
plt.legend()
plt.show()
